# MATH7016 Deep Learning
### Coursework 2025

In [22]:
import pandas as pd
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from sklearn.model_selection import KFold
from keras.layers import Activation
from keras.utils import get_custom_objects
from sklearn.preprocessing import StandardScaler

tf.random.set_seed(42)

In [23]:
# Read training and test datasets
train_df = pd.read_csv('DL-2025-CW-data/Data_A.csv')
test_df = pd.read_csv('DL-2025-CW-data/Data_B_nolabels.csv')

# Define column names
columns_names = ['target']
level_cols=[]
for i in range(4):
    level_cols +=[f'ask_price{i+1}',f'ask_size{i+1}',f'bid_price{i+1}',f'bid_size{i+1}']
change_cols =  []
for i in range(5):
    change_cols+=[f'midprice_change{i+1}']
columns_names+=level_cols+change_cols
train_df.columns = columns_names
test_df.columns = columns_names[1:]

test_df.head()

,ask_price1,ask_size1,bid_price1,bid_size1,ask_price2,ask_size2,bid_price2,bid_size2,ask_price3,ask_size3,...,bid_size3,ask_price4,ask_size4,bid_price4,bid_size4,midprice_change1,midprice_change2,midprice_change3,midprice_change4,midprice_change5
0,696400.0,16,696000.0,12,696500.0,57,695900.0,118,696600.0,100,...,262,696700.0,150,695700.0,104,1,0,1,0,1
1,740800.0,2,740400.0,20,741000.0,60,740200.0,27,741200.0,156,...,31,741300.0,200,740000.0,170,0,1,0,0,1
2,730900.0,1,730200.0,230,731000.0,111,730100.0,86,731100.0,42,...,136,731200.0,100,729900.0,132,1,1,0,0,1
3,630600.0,100,630300.0,69,630700.0,110,630200.0,2,630800.0,219,...,1,630900.0,101,630000.0,104,0,1,0,0,0
4,851100.0,579,850300.0,25,851200.0,17,850100.0,287,851400.0,307,...,270,851500.0,223,849900.0,72,1,0,1,0,1


In [24]:
def mid_price(df,  bid_p, ask_p):
    return (df[bid_p] + df[ask_p])/2

def calc_wap(df, bid_p, ask_p, bid_s, ask_s):
    return (df[bid_p] * df[ask_s] + df[ask_p] * df[bid_s]) / (df[bid_s] + df[ask_s])


In [25]:
def preprocessor(df):
    # Step 1: Calculate price & volume features
    for i in range(1,5):
        df[f"wap{i}"] = calc_wap(df, f"bid_price{i}", f"ask_price{i}", f"bid_size{i}", f"ask_size{i}")
        #df[f"wmp{i}"] = calc_wmp(df, f"bid_price{i}", f"ask_price{i}", f"bid_size{i}", f"ask_size{i}")
        df[f"price_spread{i}"] = (df[f"ask_price{i}"] - df[f"bid_price{i}"]) / ((df[f"ask_price{i}"] + df[f"bid_price{i}"]))
        df[f'midprice{i}']=mid_price(df,f'bid_price{i}',f'ask_price{i}')
 
        df[f"order_imbalance{i}"] = (df[f"bid_size{i}"] - df[f"ask_size{i}"]) / (df[f"bid_size{i}"] + df[f"ask_size{i}"])
        df[f'bid_ask_ratio{i}'] = df[f'bid_size{i}'] / (df[f'bid_size{i}'] + df[f'ask_size{i}'])
        
    # Price features
    df['bid_depth_ratio'] = df['bid_size1'] / (df[['bid_size1','bid_size2','bid_size3','bid_size4']].sum(axis=1))
    df['ask_depth_ratio'] = df['ask_size1'] / (df[['ask_size1','ask_size2','ask_size3','ask_size4']].sum(axis=1))

    # Volume features
    df["total_volume"] = df[["ask_size1", "ask_size2", "bid_size1", "bid_size2"]].sum(axis=1)
    df["volume_imbalance"] = (
        (df["ask_size1"] + df["ask_size2"]) - (df["bid_size1"] + df["bid_size2"])
    ).abs()

    # Order imbalances 
    df['bid_vol_ratio'] = df[['bid_size1','bid_size2']].sum(axis=1) / df[['bid_size1','bid_size2','ask_size1','ask_size2']].sum(axis=1)
    df['ask_vol_ratio'] = 1 - df['bid_vol_ratio']
    
    #df = df.drop(columns=level_cols)
    
    return df


In [26]:
train_nn = preprocessor(train_df)
test_nn = preprocessor(test_df)

In [27]:
train_nn 

,target,ask_price1,ask_size1,bid_price1,bid_size1,ask_price2,ask_size2,bid_price2,bid_size2,ask_price3,...,price_spread4,midprice4,order_imbalance4,bid_ask_ratio4,bid_depth_ratio,ask_depth_ratio,total_volume,volume_imbalance,bid_vol_ratio,ask_vol_ratio
0,0,650400.0,501,650200.0,106,650500.0,245,650100.0,259,650600.0,...,0.000615,650300.0,0.325359,0.662679,0.109278,0.484058,1111,381,0.328533,0.671467
1,1,636200.0,153,635800.0,150,636300.0,100,635700.0,15,636400.0,...,0.000786,636000.0,0.313725,0.656863,0.315126,0.301181,418,88,0.394737,0.605263
2,0,724800.0,4,724500.0,14,724900.0,50,724300.0,312,725100.0,...,0.000759,724650.0,-0.742529,0.128736,0.029046,0.006861,380,272,0.857895,0.142105
3,0,622900.0,110,622700.0,100,623000.0,523,622600.0,300,623100.0,...,0.000642,622800.0,-0.503106,0.248447,0.117786,0.065632,1033,233,0.387222,0.612778
4,1,620100.0,374,619900.0,110,620200.0,495,619800.0,210,620300.0,...,0.000645,620000.0,-0.512195,0.243902,0.107738,0.269258,1189,549,0.269134,0.730866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199994,0,429000.0,400,428800.0,500,429100.0,300,428700.0,700,429200.0,...,0.000933,428900.0,0.142857,0.571429,0.221729,0.210526,1900,500,0.631579,0.368421
199995,1,413600.0,100,413500.0,500,413700.0,611,413400.0,457,413800.0,...,0.000846,413550.0,0.454545,0.727273,0.231803,0.070872,1668,246,0.573741,0.426259
199996,0,381900.0,401,381800.0,100,382000.0,705,381700.0,800,382100.0,...,0.000917,381850.0,0.000000,0.500000,0.058824,0.179821,2006,206,0.448654,0.551346
199997,1,443000.0,100,442900.0,29,443100.0,300,442800.0,100,443200.0,...,0.000790,442950.0,-0.600000,0.200000,0.039781,0.066667,529,271,0.243856,0.756144


In [28]:
def swish(x, beta = 1):
    return (x * K.sigmoid(beta * x))

get_custom_objects().update({'swish': Activation(swish)})

hidden_units = (128,64,32)
feature_cols = [c for c in train_nn.columns if c != 'target']
num_features = len(feature_cols)

def base_model():
  
    num_input = keras.Input(shape=(num_features,), name='num_data')
    x = keras.layers.Concatenate()([num_input])
   
    for n_hidden in hidden_units:
        x = keras.layers.Dense(n_hidden, activation='swish')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dropout(0.3)(x)
    out = keras.layers.Dense(1, activation='sigmoid', name='prediction')(x)

    model = keras.Model(
        inputs=[num_input],
        outputs=out
    )
    return model


In [29]:
train_nn = train_nn.fillna(0)
test_nn = test_nn.fillna(0)

X = train_nn[feature_cols]
y = train_nn['target']

test_predictions_nn = np.zeros(test_nn.shape[0])
scores_folds = {'NN_model': []}

# KFold 
kfolds = 5
kf = KFold(n_splits=kfolds, shuffle=True, random_state=42)

counter = 1

for train_index, val_index in kf.split(X):

    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_val = X.iloc[val_index]
    y_val = y.iloc[val_index]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    model = base_model()
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.005),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # ---- callbacks ----
    # Early stopping callback
    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=20, verbose=0,
        mode='min', restore_best_weights=True)

    # Learning rate reduction callback 
    plateau = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=7, verbose=0,
        mode='min')
    model.fit(
        [X_train_scaled],
        y_train,
        batch_size=512,
        epochs=200,
        validation_data=([X_val_scaled], y_val),
        callbacks=[es, plateau],
        shuffle=True,
        verbose=1
    )
    
    preds = model.predict([X_val_scaled]).reshape(-1)
    score = np.mean((preds.round() != y_val.values).astype(float)) 
    print(f'Fold {counter}: Accuracy = {1 - score:.5f}')
    scores_folds['NN_model'].append(1 - score)
    
    X_test = test_nn[feature_cols]
    X_test_scaled = scaler.transform(X_test)

    test_predictions_nn += model.predict([X_test_scaled]).reshape(-1) / kfolds
   
    counter += 1
print("CV folds scores:", scores_folds['NN_model'])

Epoch 1/200
313/313 [==============================] - 3s 6ms/step - loss: 0.5876 - accuracy: 0.6803 - val_loss: 0.5540 - val_accuracy: 0.6984 - lr: 0.0050
Epoch 2/200
313/313 [==============================] - 1s 5ms/step - loss: 0.5618 - accuracy: 0.6913 - val_loss: 0.5451 - val_accuracy: 0.6982 - lr: 0.0050
Epoch 3/200
313/313 [==============================] - 1s 5ms/step - loss: 0.5537 - accuracy: 0.6941 - val_loss: 0.5368 - val_accuracy: 0.7042 - lr: 0.0050
Epoch 4/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5481 - accuracy: 0.6986 - val_loss: 0.5360 - val_accuracy: 0.7075 - lr: 0.0050
Epoch 5/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5450 - accuracy: 0.7009 - val_loss: 0.5315 - val_accuracy: 0.7063 - lr: 0.0050
Epoch 6/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5426 - accuracy: 0.7034 - val_loss: 0.5284 - val_accuracy: 0.7097 - lr: 0.0050
Epoch 7/200
313/313 [==============================] - 1s 4ms/st

Epoch 1/200
313/313 [==============================] - 2s 5ms/step - loss: 0.5883 - accuracy: 0.6815 - val_loss: 0.5548 - val_accuracy: 0.6993 - lr: 0.0050
Epoch 2/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5609 - accuracy: 0.6929 - val_loss: 0.5460 - val_accuracy: 0.6996 - lr: 0.0050
Epoch 3/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5523 - accuracy: 0.6961 - val_loss: 0.5358 - val_accuracy: 0.7054 - lr: 0.0050
Epoch 4/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5478 - accuracy: 0.6990 - val_loss: 0.5315 - val_accuracy: 0.7086 - lr: 0.0050
Epoch 5/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5440 - accuracy: 0.7026 - val_loss: 0.5293 - val_accuracy: 0.7103 - lr: 0.0050
Epoch 6/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5422 - accuracy: 0.7032 - val_loss: 0.5293 - val_accuracy: 0.7089 - lr: 0.0050
Epoch 7/200
313/313 [==============================] - 1s 4ms/st

Epoch 1/200
313/313 [==============================] - 2s 4ms/step - loss: 0.5813 - accuracy: 0.6839 - val_loss: 0.5561 - val_accuracy: 0.6942 - lr: 0.0050
Epoch 2/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5578 - accuracy: 0.6941 - val_loss: 0.5436 - val_accuracy: 0.6983 - lr: 0.0050
Epoch 3/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5499 - accuracy: 0.6971 - val_loss: 0.5376 - val_accuracy: 0.7037 - lr: 0.0050
Epoch 4/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5448 - accuracy: 0.7022 - val_loss: 0.5345 - val_accuracy: 0.7071 - lr: 0.0050
Epoch 5/200
313/313 [==============================] - 3s 10ms/step - loss: 0.5420 - accuracy: 0.7036 - val_loss: 0.5336 - val_accuracy: 0.7068 - lr: 0.0050
Epoch 6/200
313/313 [==============================] - 1s 5ms/step - loss: 0.5402 - accuracy: 0.7050 - val_loss: 0.5319 - val_accuracy: 0.7090 - lr: 0.0050
Epoch 7/200
313/313 [==============================] - 1s 4ms/s

Epoch 1/200
313/313 [==============================] - 3s 7ms/step - loss: 0.5863 - accuracy: 0.6824 - val_loss: 0.5537 - val_accuracy: 0.6967 - lr: 0.0050
Epoch 2/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5589 - accuracy: 0.6922 - val_loss: 0.5450 - val_accuracy: 0.6981 - lr: 0.0050
Epoch 3/200
313/313 [==============================] - 1s 5ms/step - loss: 0.5501 - accuracy: 0.6972 - val_loss: 0.5390 - val_accuracy: 0.7029 - lr: 0.0050
Epoch 4/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5453 - accuracy: 0.7005 - val_loss: 0.5380 - val_accuracy: 0.7028 - lr: 0.0050
Epoch 5/200
313/313 [==============================] - 2s 7ms/step - loss: 0.5417 - accuracy: 0.7024 - val_loss: 0.5335 - val_accuracy: 0.7065 - lr: 0.0050
Epoch 6/200
313/313 [==============================] - 2s 8ms/step - loss: 0.5404 - accuracy: 0.7040 - val_loss: 0.5301 - val_accuracy: 0.7096 - lr: 0.0050
Epoch 7/200
313/313 [==============================] - 2s 5ms/st

Epoch 1/200
313/313 [==============================] - 2s 4ms/step - loss: 0.5859 - accuracy: 0.6814 - val_loss: 0.5602 - val_accuracy: 0.6960 - lr: 0.0050
Epoch 2/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5591 - accuracy: 0.6928 - val_loss: 0.5494 - val_accuracy: 0.6984 - lr: 0.0050
Epoch 3/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5504 - accuracy: 0.6963 - val_loss: 0.5414 - val_accuracy: 0.7029 - lr: 0.0050
Epoch 4/200
313/313 [==============================] - 1s 4ms/step - loss: 0.5454 - accuracy: 0.6996 - val_loss: 0.5382 - val_accuracy: 0.7068 - lr: 0.0050
Epoch 5/200
313/313 [==============================] - 2s 5ms/step - loss: 0.5425 - accuracy: 0.7031 - val_loss: 0.5359 - val_accuracy: 0.7078 - lr: 0.0050
Epoch 6/200
313/313 [==============================] - 2s 5ms/step - loss: 0.5397 - accuracy: 0.7044 - val_loss: 0.5346 - val_accuracy: 0.7101 - lr: 0.0050
Epoch 7/200
313/313 [==============================] - 2s 6ms/st

In [30]:
final_pred = (test_predictions_nn > 0.5).astype(int)
np.savetxt("02205857_Wang.txt", final_pred, fmt="%d")